In [ ]:
from yugiquery import *

init_notebook_mode(all_interactive=True)

header("Cards")

---

Table of Contents <a class="jp-toc-ignore"></a>
=================

*   [1  Data aquisition](#Data-aquisition)
    *   [1.1  Fetch online data](#Fetch-online-data)
    *   [1.2  Merge data](#Merge-data)
*   [2  Check changes](#Check-changes)
    *   [2.1  Load previous data](#Load-previous-data)
    *   [2.2  Generate changelog](#Generate-changelog)
    *   [2.3  Save data](#Save-data)
*   [3  Data visualization](#Data-visualization)
    *   [3.1  Full data](#Full-data)
    *   [3.2  Card types](#Card-types)
    *   [3.3  Monsters](#Monsters)
        *   [3.3.1  Attributes](#Attributes)
        *   [3.3.2  Primary types](#Primary-types)
            *   [3.3.2.1  Has effect discrimination](#Has-effect-discrimination)
            *   [3.3.2.2  Is pendulum discrimination](#Is-pendulum-discrimination)
            *   [3.3.2.3  By attribute](#By-attribute)
        *   [3.3.3  Secondary types](#Secondary-types)
            *   [3.3.3.1  By attribute](#By-attribute)
            *   [3.3.3.2  By secondary type](#By-secondary-type)
        *   [3.3.4  Monster types](#Monster-types)
            *   [3.3.4.1  By Attribute](#By-Attribute)
            *   [3.3.4.2  By primary type](#By-primary-type)
            *   [3.3.4.3  By secondary type](#By-secondary-type)
        *   [3.3.5  ATK](#ATK)
        *   [3.3.6  DEF](#DEF)
        *   [3.3.7  Level/Rank](#Level/Rank)
            *   [3.3.7.1  ATK statistics](#ATK-statistics)
            *   [3.3.7.2  DEF statistics](#DEF-statistics)
        *   [3.3.8  Pendulum scale](#Pendulum-scale)
            *   [3.3.8.1  ATK statistics](#ATK-statistics)
            *   [3.3.8.2  DEF statistics](#DEF-statistics)
            *   [3.3.8.3  Level/Rank statistics](#Level/Rank-statistics)
        *   [3.3.9  Link](#Link)
            *   [3.3.9.1  ATK statistics](#ATK-statistics)
        *   [3.3.10  Link Arrows](#Link-Arrows)
            *   [3.3.10.1  By combination](#By-combination)
            *   [3.3.10.2  By unique](#By-unique)
            *   [3.3.10.3  By link](#By-link)
    *   [3.4  Spell & Trap](#Spell-&-Trap)
        *   [3.4.1  Properties](#Properties)
    *   [3.5  Effect type](#Effect-type)
        *   [3.5.1  Card type discrimination](#Card-type-discrimination)
    *   [3.6  Archseries](#Archseries)
        *   [3.6.1  By card type](#By-card-type)
        *   [3.6.2  By primary type](#By-primary-type)
        *   [3.6.3  By secondary type](#By-secondary-type)
        *   [3.6.4  By monster type](#By-monster-type)
        *   [3.6.5  By property](#By-property)
    *   [3.7  Artworks](#Artworks)
        *   [3.7.1  By card type](#By-card-type)
        *   [3.7.2  By primary type](#By-primary-type)
    *   [3.8  Errata](#Errata)
        *   [3.8.1  By card type](#By-card-type)
        *   [3.8.2  By primary type](#By-primary-type)
        *   [3.8.3  By artwork](#By-artwork)
    *   [3.9  TCG & OCG status](#TCG-&-OCG-status)
        *   [3.9.1  TGC status](#TGC-status)
            *   [3.9.1.1  By card type](#By-card-type)
            *   [3.9.1.2  By monster type](#By-monster-type)
            *   [3.9.1.3  By archseries](#By-archseries)
        *   [3.9.2  OCG status](#OCG-status)
            *   [3.9.2.1  By card type](#By-card-type)
            *   [3.9.2.2  By monster type](#By-monster-type)
            *   [3.9.2.3  By archseries](#By-archseries)
        *   [3.9.3  TCG vs. OCG status](#TCG-vs.-OCG-status)
*   [4  Extras](#Extras)
    *   [4.1 Multiple secondary types](#Multiple-secondary-types)
    *   [4.2 Not yet released](#Not-yet-released)
    *   [4.3 Counters and Tokens](#Counters-and-Tokens)
*   [5  Epilogue](#Epilogue)
    *   [5.1  HTML export](#HTML-export)
<!-- *   [5.2  Git](#Git) -->

# Data aquisition

## Fetch online data

In [ ]:
# Timestamp
timestamp = arrow.utcnow()

In [ ]:
# Fetch Monster
monster_df = fetch_monster()

In [ ]:
# Fetch Spell
spell_df = fetch_st(st="Spell")

# Fetch Trap
trap_df = fetch_st(st="Trap")

In [ ]:
# Fetch Token
token_df = fetch_token()

# Fetch Counter
counter_df = fetch_counter()

# Fetch errata
errata_df = fetch_errata()

## Merge data

In [ ]:
# Merge errata
monster_df = merge_errata(monster_df, errata_df)
spell_df = merge_errata(spell_df, errata_df)
trap_df = merge_errata(trap_df, errata_df)
token_df = merge_errata(token_df, errata_df)
counter_df = merge_errata(counter_df, errata_df)

# Add counters to tokens data frame
token_df = (
    pd.concat([token_df, counter_df], ignore_index=True, axis=0)
    .drop_duplicates()
    .sort_values("Name", ignore_index=True)
)

# Create combined spell and trap data frame
st_df = pd.concat([spell_df, trap_df], ignore_index=True, axis=0).sort_values(
    "Name", ignore_index=True
)

# Create combined monster, spell and trap data frames
cards_df = pd.concat([st_df, monster_df], ignore_index=True, axis=0).sort_values(
    "Name", ignore_index=True
)

# Create combined data frames with full data
full_df = pd.concat([cards_df, token_df], ignore_index=True, axis=0).sort_values(
    "Name", ignore_index=True
)

print("Data merged")

# Check changes

## Load previous data

In [ ]:
# Get latest file if exist
tuple_cols = [
    "Effect type",
    "Secondary type",
    "Link Arrows",
    "Archseries",
    "Artwork",
    "Errata",
]
previous_df, previous_ts = load_corrected_latest("cards", tuple_cols)

if previous_df is not None:
    previous_df = previous_df.astype(
        full_df[previous_df.columns.intersection(full_df.columns)].dtypes.to_dict()
    )

## Generate changelog

In [ ]:
if previous_df is None:
    changelog = None
    print("Skipped")
else:
    changelog = generate_changelog(previous_df, full_df, col="Name")
    if not changelog.empty:
        display(changelog)
        changelog.to_csv(
            f'../data/{make_filename(report = "cards", timestamp = timestamp, previous_timestamp = previous_ts)}',
            index=True,
        )
        print("Changelog saved")

## Save data

In [ ]:
if changelog is not None and changelog.empty:
    print("No changes. New data not saved")
else:
    full_df.to_csv(
        f'../data/{make_filename(report = "cards", timestamp = timestamp)}', index=False
    )
    print("Data saved")

# Data visualization

## Full data

In [ ]:
full_df

Full data available to download [here](../data)

## Card types

In [ ]:
print("Total number of card types:", cards_df["Card type"].nunique())

In [ ]:
card_type_colors = [colors_dict[i] for i in full_df["Card type"].value_counts().index]
cards_df["Card type"].value_counts().plot.bar(
    figsize=(18, 6), grid=True, rot=0, color=card_type_colors
)
plt.show()

## Monsters

### Attributes

In [ ]:
print("Total number of attributes:", monster_df["Attribute"].nunique())

In [ ]:
monster_df.drop(columns=["Card type", "Page name", "Page URL"]).groupby(
    "Attribute"
).nunique()

In [ ]:
attribute_colors = [
    colors_dict[i] for i in monster_df["Attribute"].value_counts().index
]
monster_df["Attribute"].value_counts().plot.bar(
    figsize=(18, 6), grid=True, rot=0, color=attribute_colors
)
plt.show()

### Primary types

In [ ]:
print("Total number of primary types:", monster_df["Primary type"].nunique())

In [ ]:
monster_df.drop(columns=["Card type", "Page name", "Page URL"]).groupby(
    "Primary type"
).nunique()

#### Has effect discrimination

In [ ]:
effect = pd.crosstab(
    monster_df["Primary type"],
    pd.isna(monster_df["Effect type"]),
    rownames=["Primary type"],
    colnames=["Has effect"],
).rename(columns={True: "No Effect", False: "Effect"})
effect

In [ ]:
monster_type_colors = {
    "No Effect": colors_dict["Normal Monster"],
    "Effect": [colors_dict[i] for i in effect.index],
}
effect.plot.bar(
    figsize=(18, 6),
    stacked=True,
    grid=True,
    rot=0,
    legend=True,
    color=monster_type_colors,
)
# plt.yscale('log')
plt.show()

Obs: Normal monster can have effect if it is pendulum

#### Is pendulum discrimination

In [ ]:
pendulum = pd.crosstab(
    monster_df["Primary type"],
    pd.isna(monster_df["Pendulum Scale"]),
    rownames=["Primary type"],
    colnames=["Is Pendulum"],
).rename(columns={True: "Not Pendulum", False: "Pendulum"})
pendulum

In [ ]:
monster_type_colors_b = {
    "Pendulum": colors_dict["Pendulum Monster"],
    "Not Pendulum": [colors_dict[i] for i in pendulum.index],
}
pendulum.plot.bar(
    figsize=(18, 6),
    stacked=True,
    grid=True,
    rot=0,
    color=monster_type_colors_b,
    legend=True,
    title="Primary types - Is pendulum",
)
plt.show()

#### By attribute

In [ ]:
primmary_crosstab = pd.crosstab(cards_df["Primary type"], cards_df["Attribute"])
primmary_crosstab

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(
    primmary_crosstab.T,
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
    norm=mc.LogNorm(),
)
plt.show()

### Secondary types

In [ ]:
exploded_secondary_type = monster_df.explode("Secondary type")
print(
    "Total number of secondary types:",
    exploded_secondary_type["Secondary type"].nunique(),
)

In [ ]:
exploded_secondary_type.drop(
    columns=["Card type", "Link", "Link Arrows", "Page name", "Page URL"]
).groupby("Secondary type").nunique()

In [ ]:
secondary_type_colors = colors_dict["Effect Monster"]
exploded_secondary_type["Secondary type"].value_counts().plot.bar(
    figsize=(18, 6),
    stacked=True,
    grid=True,
    rot=0,
    color=secondary_type_colors,
    legend=False,
)
plt.show()

#### By attribute

In [ ]:
secondary_crosstab = pd.crosstab(
    exploded_secondary_type["Secondary type"], exploded_secondary_type["Attribute"]
)
secondary_crosstab

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(
    secondary_crosstab[secondary_crosstab > 0],
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
)
plt.show()

#### By secondary type

In [ ]:
secondary_crosstab_b = pd.crosstab(
    exploded_secondary_type["Primary type"],
    exploded_secondary_type["Secondary type"],
    margins=True,
)
secondary_crosstab_b

In [ ]:
plt.figure(figsize=(10, 7))
sns.heatmap(
    secondary_crosstab_b,
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
    norm=mc.LogNorm(),
)
plt.show()

### Monster types

In [ ]:
print("Total number of monster types:", monster_df["Monster type"].nunique())

In [ ]:
monster_df.drop(columns=["Card type", "Page name", "Page URL"]).groupby(
    "Monster type"
).nunique()

In [ ]:
monster_type_colors = colors_dict["Monster Card"]
monster_df["Monster type"].value_counts().plot.bar(
    figsize=(18, 6), grid=True, rot=45, color=monster_type_colors
)
plt.show()

#### By Attribute

In [ ]:
monster_crosstab = pd.crosstab(cards_df["Monster type"], cards_df["Attribute"])
monster_crosstab

In [ ]:
plt.figure(figsize=(20, 5))
sns.heatmap(
    monster_crosstab[monster_crosstab > 0].T,
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
    norm=mc.LogNorm(),
)
plt.show()

#### By primary type

In [ ]:
monster_crosstab_b = pd.crosstab(
    cards_df["Monster type"], cards_df["Primary type"], dropna=False
)
monster_crosstab_b

In [ ]:
plt.figure(figsize=(20, 5))
sns.heatmap(
    monster_crosstab_b[monster_crosstab_b > 0].T,
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
    norm=mc.LogNorm(),
)
plt.show()

#### By secondary type

In [ ]:
monster_crosstab_c = pd.crosstab(
    exploded_secondary_type["Monster type"],
    exploded_secondary_type["Secondary type"],
    dropna=False,
)
monster_crosstab_c

In [ ]:
plt.figure(figsize=(20, 5))
sns.heatmap(
    monster_crosstab_c[monster_crosstab_c > 0].T,
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
    norm=mc.LogNorm(),
)
plt.show()

### ATK

In [ ]:
print("Total number of ATK values:", monster_df["ATK"].nunique())

In [ ]:
monster_df.drop(columns=["Card type", "Page name", "Page URL"]).groupby(
    "ATK"
).nunique().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce"))

In [ ]:
atk_colors = colors_dict["Monster Card"]
monster_df["ATK"].value_counts().sort_index(
    key=lambda x: pd.to_numeric(x, errors="coerce")
).plot.bar(figsize=(18, 6), grid=True, color=atk_colors)
plt.show()

### DEF

In [ ]:
print("Total number of DEF values:", monster_df["DEF"].nunique())

In [ ]:
monster_df.drop(columns=["Card type", "Page name", "Page URL"]).groupby(
    "DEF"
).nunique().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce"))

In [ ]:
def_colors = colors_dict["Monster Card"]
monster_df["DEF"].value_counts().sort_index(
    key=lambda x: pd.to_numeric(x, errors="coerce")
).plot.bar(figsize=(18, 6), grid=True, color=def_colors)
plt.show()

### Level/Rank

In [ ]:
monster_df.drop(
    columns=["Card type", "Link", "Link Arrows", "Page name", "Page URL"]
).groupby("Level/Rank").nunique().sort_index(
    key=lambda x: pd.to_numeric(x, errors="coerce")
)

In [ ]:
stars_colors = colors_dict["Level"]
monster_df["Level/Rank"].value_counts().sort_index(
    key=lambda x: pd.to_numeric(x, errors="coerce")
).plot.bar(figsize=(18, 6), grid=True, rot=0, color=stars_colors)
plt.show()

#### ATK statistics

In [ ]:
monster_df[["Level/Rank", "ATK"]].apply(pd.to_numeric, errors="coerce").dropna().astype(
    int
).groupby("Level/Rank").describe().round(1)

#### DEF statistics

In [ ]:
monster_df[["Level/Rank", "DEF"]].apply(pd.to_numeric, errors="coerce").dropna().astype(
    int
).groupby("Level/Rank").describe().round(1)

### Pendulum scale

In [ ]:
monster_df.drop(
    columns=["Card type", "Link", "Link Arrows", "Page name", "Page URL"]
).groupby("Pendulum Scale").nunique().sort_index(
    key=lambda x: pd.to_numeric(x, errors="coerce")
)

In [ ]:
scales_colors = colors_dict["Pendulum Monster"]
monster_df["Pendulum Scale"].value_counts().sort_index(
    key=lambda x: pd.to_numeric(x, errors="coerce")
).plot.bar(figsize=(18, 6), grid=True, rot=0, color=scales_colors)
plt.show()

#### ATK statistics

In [ ]:
monster_df[["Pendulum Scale", "ATK"]].apply(
    pd.to_numeric, errors="coerce"
).dropna().astype(int).groupby("Pendulum Scale").describe().round(1)

#### DEF statistics

In [ ]:
monster_df[["Pendulum Scale", "DEF"]].apply(
    pd.to_numeric, errors="coerce"
).dropna().astype(int).groupby("Pendulum Scale").describe().round(1)

#### Level/Rank statistics

In [ ]:
monster_df[["Pendulum Scale", "Level/Rank"]].apply(
    pd.to_numeric, errors="coerce"
).dropna().astype(int).groupby("Pendulum Scale").describe().round(1)

### Link

In [ ]:
monster_df.drop(
    columns=[
        "Card type",
        "Primary type",
        "Secondary type",
        "Level/Rank",
        "DEF",
        "Pendulum Scale",
        "Page name",
        "Page URL",
    ]
).groupby("Link").nunique().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce"))

In [ ]:
link_colors = colors_dict["Link Monster"]
monster_df["Link"].value_counts().sort_index(
    key=lambda x: pd.to_numeric(x, errors="coerce")
).plot.bar(figsize=(18, 6), grid=True, rot=0, color=link_colors)
plt.show()

#### ATK statistics

In [ ]:
monster_df[["Link", "ATK"]].apply(pd.to_numeric, errors="coerce").dropna().astype(
    int
).groupby("Link").describe().round(1)

### Link Arrows

#### By combination

In [ ]:
print("Total number of link arrow combinations:", monster_df["Link Arrows"].nunique())

In [ ]:
monster_df.drop(
    columns=[
        "Card type",
        "Primary type",
        "Level/Rank",
        "Pendulum Scale",
        "Link",
        "Secondary type",
        "DEF",
        "Page name",
        "Page URL",
    ]
).groupby("Link Arrows").nunique()

In [ ]:
arrows_colors = colors_dict["Link Monster"]
monster_df["Link Arrows"].value_counts().plot.barh(
    figsize=(10, 20), grid=True, color=arrows_colors, title="Link arrows combinations"
)
plt.show()

#### By unique

In [ ]:
monster_df[monster_df["Link Arrows"].notna()].drop(
    columns=[
        "Card type",
        "Primary type",
        "Level/Rank",
        "Pendulum Scale",
        "Secondary type",
        "DEF",
        "Page name",
        "Page URL",
    ]
).explode("Link Arrows").groupby("Link Arrows").nunique()

In [ ]:
arrow_plot(monster_df["Link Arrows"].explode("Link Arrows"))

#### By link

In [ ]:
arrow_per_link = monster_df[["Link Arrows", "Link"]].explode("Link Arrows").dropna()
arrow_crosstab = pd.crosstab(arrow_per_link["Link Arrows"], arrow_per_link["Link"])
arrow_crosstab

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(
    arrow_crosstab[arrow_crosstab > 0].T,
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
    norm=mc.LogNorm(),
)
plt.show()

## Spell & Trap

### Properties

In [ ]:
print("Total number of properties:", st_df["Property"].nunique())

In [ ]:
st_df.drop(columns=["Card type", "Page name", "Page URL"]).groupby("Property").nunique()

In [ ]:
st_colors = [
    colors_dict[i]
    for i in cards_df[["Card type", "Property"]]
    .value_counts()
    .index.get_level_values(0)
]
st_df["Property"].value_counts().plot.bar(
    figsize=(18, 6), grid=True, rot=45, color=st_colors
)
plt.show()

## Effect type

In [ ]:
print("Total number of effect types:", full_df["Effect type"].explode().nunique())

In [ ]:
full_df.explode("Effect type").groupby("Effect type").nunique()

### Card type discrimination

In [ ]:
type_diff = (
    full_df[["Card type", "Effect type"]]
    .explode("Effect type")
    .value_counts()
    .unstack(0)
    .fillna(0)
    .astype(int)
)
type_diff

In [ ]:
type_diff_colors = {
    type: colors_dict[type] for type in full_df["Card type"].dropna().unique()
}
type_diff.plot.bar(
    figsize=(18, 6), stacked=True, grid=True, rot=45, color=type_diff_colors
)
plt.show()

## Archseries

In [ ]:
exploded_archseries = cards_df.explode("Archseries")
print("Total number of Archseries:", exploded_archseries["Archseries"].nunique())

In [ ]:
exploded_archseries.groupby("Archseries").nunique()

In [ ]:
exploded_archseries["Archseries"].value_counts().plot.barh(
    figsize=(10, 200), grid=True, title="Archtypes/Series"
)
plt.show()

### By card type

In [ ]:
archseries_crosstab = pd.crosstab(
    exploded_archseries["Archseries"], exploded_archseries["Card type"], margins=True
)
archseries_crosstab

### By primary type

In [ ]:
archseries_crosstab_b = pd.crosstab(
    exploded_archseries["Archseries"], exploded_archseries["Primary type"], margins=True
)
archseries_crosstab_b



### By secondary type

In [ ]:
exploded_archseries_secondary_type = exploded_archseries[
    ["Archseries", "Secondary type"]
].explode("Secondary type")
archseries_crosstab_c = pd.crosstab(
    exploded_archseries_secondary_type["Archseries"],
    exploded_archseries_secondary_type["Secondary type"],
    margins=True,
)
archseries_crosstab_c

### By monster type

In [ ]:
archseries_crosstab_d = pd.crosstab(
    exploded_archseries["Archseries"], exploded_archseries["Monster type"], margins=True
)
archseries_crosstab_d

### By property

In [ ]:
archseries_crosstab_e = pd.crosstab(
    exploded_archseries["Archseries"], exploded_archseries["Property"], margins=True
)
archseries_crosstab_e

## Artworks

In [ ]:
print(
    "Total number of cards with edited or alternate artworks:",
    cards_df["Artwork"].count(),
)

In [ ]:
cards_df[["Name", "Password", "TCG status", "OCG status", "Artwork"]][
    cards_df["Artwork"].notna()
]

In [ ]:
artwork_value_counts = cards_df["Artwork"].value_counts()
plt.figure(figsize=(20, 8))
plt.title("Artworks")
venn2(
    subsets=(
        artwork_value_counts[("Alternate",)],
        artwork_value_counts[("Edited",)],
        artwork_value_counts[("Alternate", "Edited")],
    ),
    set_labels=("Alternate artwork", "Edited artwork"),
)
plt.show()

### By card type

In [ ]:
artwork_crosstab = pd.crosstab(cards_df["Artwork"], cards_df["Card type"])
artwork_crosstab

### By primary type

In [ ]:
artwork_crosstab_b = pd.crosstab(cards_df["Artwork"], cards_df["Primary type"])
artwork_crosstab_b

More granularity is unnecessary

## Errata

In [ ]:
print("Total number of cards with errata:", cards_df["Errata"].count())

In [ ]:
cards_df[["Name", "Password", "TCG status", "OCG status", "Errata"]][
    cards_df["Errata"].notna()
]

In [ ]:
errata_counts = (
    cards_df.groupby("Errata").nunique().sort_values("Name", ascending=False)
)
errata_counts

In [ ]:
plt.figure(figsize=(20, 8))
plt.title("Errata")
sorted_errata_name_counts = (
    errata_counts["Name"]
    .drop(("Any",))
    .sort_index(key=lambda x: [(len(i), i) for i in x])
)
venn2(
    subsets=sorted_errata_name_counts,
    set_labels=sorted_errata_name_counts.index[:-1].str[0],
)
plt.show()

### By card type

In [ ]:
errata_crosstab = pd.crosstab(cards_df["Errata"], cards_df["Card type"])
errata_crosstab.sort_values(by=errata_crosstab.columns.tolist(), ascending=False)

### By primary type

In [ ]:
errata_crosstab_b = pd.crosstab(cards_df["Errata"], cards_df["Primary type"])
errata_crosstab_b.sort_values(by=errata_crosstab_b.columns.tolist(), ascending=False)

More granularity is unnecessary

### By artwork

In [ ]:
errata_crosstab_c = pd.crosstab(cards_df["Artwork"], cards_df["Errata"])
errata_crosstab_c.sort_values(by=errata_crosstab_c.columns.tolist(), ascending=False)

## TCG & OCG status

### TGC status

In [ ]:
print("Total number of TCG status:", cards_df["TCG status"].nunique())

In [ ]:
cards_df.drop(columns=["Page name", "Page URL"]).groupby(
    "TCG status", dropna=False
).nunique()

In [ ]:
cards_df["TCG status"].value_counts(dropna=False).plot.bar(
    figsize=(18, 6), logy=True, grid=True, rot=45
)
plt.show()

#### By card type

In [ ]:
# Remove unlimited
tcg_crosstab = pd.crosstab(cards_df["Card type"], cards_df["TCG status"]).drop(
    ["Unlimited"], axis=1
)
tcg_crosstab

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(
    tcg_crosstab[tcg_crosstab > 0],
    annot=True,
    fmt="g",
    cmap="viridis",
    norm=mc.LogNorm(),
)
plt.show()

#### By monster type

In [ ]:
# Remove unlimited
tcg_crosstab_b = pd.crosstab(cards_df["Monster type"], cards_df["TCG status"]).drop(
    ["Unlimited"], axis=1
)
tcg_crosstab_b

In [ ]:
plt.figure(figsize=(20, 5))
sns.heatmap(
    tcg_crosstab_b[tcg_crosstab_b > 0].T,
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
)
plt.show()

#### By archseries

In [ ]:
# Remove unlimited
tcg_crosstab_c = pd.crosstab(
    exploded_archseries["Archseries"].where(
        exploded_archseries["OCG status"] != "Unlimited"
    ),
    exploded_archseries["TCG status"],
    margins=True,
)
tcg_crosstab_c

### OCG status

In [ ]:
print("Total number of OCG status:", cards_df["OCG status"].nunique())

In [ ]:
cards_df.drop(columns=["Page name", "Page URL"]).groupby(
    "OCG status", dropna=False
).nunique()

In [ ]:
cards_df["OCG status"].value_counts(dropna=False).plot.bar(
    figsize=(18, 6), logy=True, grid=True, rot=45
)
plt.show()

#### By card type

In [ ]:
# Remove unlimited
ocg_crosstab = pd.crosstab(cards_df["Card type"], cards_df["OCG status"]).drop(
    ["Unlimited"], axis=1
)
ocg_crosstab

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(ocg_crosstab[ocg_crosstab > 0], annot=True, fmt="g", cmap="viridis")
plt.show()

#### By monster type

In [ ]:
ocg_crosstab_b = pd.crosstab(cards_df["Monster type"], cards_df["OCG status"]).drop(
    ["Unlimited"], axis=1
)
ocg_crosstab_b

In [ ]:
plt.figure(figsize=(20, 5))
sns.heatmap(
    ocg_crosstab_b[ocg_crosstab_b > 0].T,
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
)
plt.show()

#### By archseries

In [ ]:
# Remove unlimited
ocg_crosstab_c = pd.crosstab(
    exploded_archseries["Archseries"].where(
        exploded_archseries["OCG status"] != "Unlimited"
    ),
    exploded_archseries["OCG status"],
    margins=True,
)
ocg_crosstab_c

### TCG vs. OCG status

In [ ]:
cg_crosstab = pd.crosstab(
    cards_df["OCG status"], cards_df["TCG status"], dropna=False, margins=False
)
cg_crosstab

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(
    cg_crosstab[cg_crosstab > 0],
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
    norm=mc.LogNorm(),
)
plt.show()

# Extras

## Multiple secondary types

In [ ]:
cards_df.dropna(subset="Secondary type", axis=0)[
    [len(x) > 1 for x in cards_df["Secondary type"].dropna()]
]

## Not yet released

In [ ]:
cards_df.loc[cards_df["OCG status"] == "Not yet released"].loc[
    full_df["TCG status"] == "Not yet released"
]

## Counters and Tokens

In [ ]:
token_df.dropna(how="all", axis=1)

# Page name differs from card name

In [ ]:
full_df[full_df["Name"] != full_df["Page name"]]

# Epilogue

In [ ]:
benchmark(report="cards", timestamp=timestamp)

In [ ]:
footer(timestamp)

## HTML export

In [ ]:
# Save notebook on disck before generating HTML report
save_notebook()

In [ ]:
! jupyter nbconvert Cards.ipynb --output-dir='../' --to=HTML --template=labdynamic --no-input

## Git

In [ ]:
commit("*[Cc]ards*", f"Cards update - {timestamp.isoformat()}")